In [109]:
# pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [110]:
import pandas as pd

import pyodbc
import sqlalchemy
import urllib

import pickle
import os.path
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

In [111]:
# Google API Check Function
def gsheet_api_check(SCOPES):
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

In [112]:
# Google Sheet Data Pull Function
def pull_sheet_data(SCOPES,SPREADSHEET_ID,DATA_TO_PULL):
    creds = gsheet_api_check(SCOPES)
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=DATA_TO_PULL).execute()
    values = result.get('values', [])
    
    if not values:
        print('No data found.')
    else:
        rows = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                  range=DATA_TO_PULL).execute()
        data = rows.get('values')
        print("COMPLETE: Data copied")
        return data

In [ ]:
# Connect to MS SQL Server
conn_string = "driver={ODBC Driver 17 for SQL Server}; server=localhost; database=winwin; TRUSTED_CONNECTION=yes;"
conn = pyodbc.connect(conn_string);
crs = conn.cursor()
conn.autocommit = True

In [ ]:
# MS SQL Execute Query
def execute_query(conn, query):
    crs = conn.cursor()
    try:
        crs.execute(query)
        print("Query Succeessful!")
    except Error as err:
        print(f"Error:'{err}'")

In [ ]:
# MS SQL Read Query
def read_query (conn, query):
    crs = conn. cursor()
    results = None
    crs.execute(query)
    results = crs.fetchall()
    return results

In [ ]:
# Create Schema
# query = 'CREATE SCHEMA Orders'
# execute_query(conn, query)

In [ ]:
# # Create Table
# query = """CREATE TABLE Orders.AshleyAll (InvoiceId NVARCHAR (50),
#   										  OrderId NVARCHAR (50),
#    									  	  TripId NVARCHAR (50),
#    										  OrderDate DATE,
#    										  Store NVARCHAR (50),
#    										  Vendor NVARCHAR (50),
#    										  Item NVARCHAR (MAX),
#    										  SKU NVARCHAR (50),
#    										  New_Item NVARCHAR (MAX),
#    										  Shiped INT,
#    										  UnitPrice NVARCHAR (50),
#    										  Ordered INT,
#    										  FreightPrice NVARCHAR (50),
#    										  TotalPrice NVARCHAR (50),
#    										  ExtendedPrice NVARCHAR (50),
#    										  Merchandise NVARCHAR (50),
#    										  Freight_Ocean NVARCHAR (50),
#    										  Fuel_Ship NVARCHAR (50),
#    										  Tax NVARCHAR (50),
#    										  Discount NVARCHAR (50),
#    										  AllTotal NVARCHAR (50));"""
# execute_query(conn, query)

Query Succeessful!


In [113]:
# Google Sheet Data Pull
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SPREADSHEET_ID = '1WPEdONpA0OVpGFV2WggOEMl0p4DMJ1a0vuPGc-eyIds'

#Pulls data from the entire spreadsheet tab.
#DATA_TO_PULL = 'AshleyAll'

#Pulls data only from the specified range of cells.
DATA_TO_PULL = 'AshleyAll!A1:U10000'

data = pull_sheet_data(SCOPES,SPREADSHEET_ID,DATA_TO_PULL)
df = pd.DataFrame(data[1:], columns=data[0])
print(df.shape)
df.head(5)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=47187434094-l8ua38hd32l2qqjq1ms6mm8boopf66er.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A53219%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets&state=gl261eT2qJqM7CUOLYstZ7q2oWSM08&access_type=offline
COMPLETE: Data copied
(1094, 21)


,InvoiceId,OrderId,TripId,OrderDate,Store,Vendor,Item,SKU,New_Item,Shiped,...,Ordered,FreightPrice,TotalPrice,ExtendedPrice,Merchandise,Freight_Ocean,Fuel_Ship,Tax,Discount,AllTotal
0,39396439,C515313,058845 06,2022-01-04,,Ashley,LOVESEAT/Claredon/Linen,1560235,"Loveseat, Claredon, Linen, 1560235",1,...,1,26.00,328.99,328.99,302.99,26.00,6.24,0.00,0.00,335.23
1,45080757,C358579,078062 04,2022-07-26,,Ashley,WEDGE/Clonmel/Chocolate,3650477,"Wedge, Clonmel, Chocolate, 3650477",1,...,1,20.50,258.50,258.50,650.00,41.00,17.22,0.00,0.00,708.22
2,38334978,D937357,035734 04,2021-11-22,,Ashley,PWR RECLINER/ADJ HEADREST,3850113,"Pwr Recliner/Adj Headrest, Game Zone, Bark, 38...",2,...,2,17.00,554.99,"1,109.98","1,075.98",34.00,8.50,0.00,0.00,"1,118.48"
3,39151374,D785138,054228 04,2021-12-22,,Ashley,ROCKER RECLINER/McCade,1010425,"Rocker Recliner, McCade, Cobblestone, 1010425",1,...,1,17.00,270.99,270.99,"2,484.89",155.00,37.20,0.00,0.00,"2,677.09"
4,39151374,D785138,054228 04,2021-12-22,,Ashley,ROCKER RECLINER/McCade,1010425,"Rocker Recliner, McCade, Cobblestone, 1010425",1,...,1,17.00,270.99,270.99,"2,484.89",155.00,37.20,0.00,0.00,"2,677.09"


In [117]:
# Use winwin database
query = 'USE winwin'
execute_query(conn, query)

Query Succeessful!


In [120]:
# Truncate the table
query = 'TRUNCATE TABLE Orders.AshleyAll'
execute_query(conn, query)

Query Succeessful!


In [121]:
# Read the Empty Table
pd.read_sql("SELECT * FROM Orders.AshleyAll", con = conn)

,InvoiceId,OrderId,TripId,OrderDate,Store,Vendor,Item,SKU,New_Item,Shiped,...,Ordered,FreightPrice,TotalPrice,ExtendedPrice,Merchandise,Freight_Ocean,Fuel_Ship,Tax,Discount,AllTotal


In [122]:
# Insert Dataframe into SQL Server:
for index, row in df.iterrows():
        crs.execute(""" INSERT INTO Orders.AshleyAll
                                (InvoiceId, OrderId, TripId, OrderDate, Store, Vendor,
                                Item, SKU, New_Item, Shiped, UnitPrice, Ordered, FreightPrice,
                                TotalPrice, ExtendedPrice, Merchandise, Freight_Ocean,
                                Fuel_Ship, Tax, Discount, AllTotal)
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
                                ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?) """,
                                row.InvoiceId, row.OrderId, row.TripId, row.OrderDate, row.Store, row.Vendor,
                                row.Item, row.SKU, row.New_Item, row.Shiped, row.UnitPrice, row.Ordered,
                                row.FreightPrice, row.TotalPrice, row.ExtendedPrice, row.Merchandise,
                                row.Freight_Ocean, row.Fuel_Ship, row.Tax, row.Discount, row.AllTotal)
conn.commit()
crs.close()

In [123]:
# Read the Full Table
df = pd.read_sql("SELECT * FROM Orders.AshleyAll", con = conn)
print(df.shape)
df.head(5)

(1094, 21)


,InvoiceId,OrderId,TripId,OrderDate,Store,Vendor,Item,SKU,New_Item,Shiped,...,Ordered,FreightPrice,TotalPrice,ExtendedPrice,Merchandise,Freight_Ocean,Fuel_Ship,Tax,Discount,AllTotal
0,39396439,C515313,058845 06,2022-01-04,,Ashley,LOVESEAT/Claredon/Linen,1560235,"Loveseat, Claredon, Linen, 1560235",1,...,1,26.00,328.99,328.99,302.99,26.00,6.24,0.00,0.00,335.23
1,45080757,C358579,078062 04,2022-07-26,,Ashley,WEDGE/Clonmel/Chocolate,3650477,"Wedge, Clonmel, Chocolate, 3650477",1,...,1,20.50,258.50,258.50,650.00,41.00,17.22,0.00,0.00,708.22
2,38334978,D937357,035734 04,2021-11-22,,Ashley,PWR RECLINER/ADJ HEADREST,3850113,"Pwr Recliner/Adj Headrest, Game Zone, Bark, 38...",2,...,2,17.00,554.99,"1,109.98","1,075.98",34.00,8.50,0.00,0.00,"1,118.48"
3,39151374,D785138,054228 04,2021-12-22,,Ashley,ROCKER RECLINER/McCade,1010425,"Rocker Recliner, McCade, Cobblestone, 1010425",1,...,1,17.00,270.99,270.99,"2,484.89",155.00,37.20,0.00,0.00,"2,677.09"
4,39151374,D785138,054228 04,2021-12-22,,Ashley,ROCKER RECLINER/McCade,1010425,"Rocker Recliner, McCade, Cobblestone, 1010425",1,...,1,17.00,270.99,270.99,"2,484.89",155.00,37.20,0.00,0.00,"2,677.09"
